In [ ]:
Mandi Data api Key = 579b464db66ec23bdd000001e3a48235641a458b502501e95c36f78e

Mandi Data file = "D:\CUDA_Experiments\Git_HUB\AgriCast360\Script\Mandi_Data.csv"

In [ ]:
Weatherbit API

Example API URL: https://api.weatherbit.io/v2.0/history/subhourly?lat=35.78&lon=78.64&start_date=2025-11-11&end_date=2025-11-12&key=API_KEY

Master API Key = de8e274a65ec45929d501a63466da6cf	

(Plan	Details	Support Level
Business Trial (Expires 2025-12-03)
1,500 req/day
1,500 historical req/day
25 years historical
Current weather + alerts + lightning
Daily forecasts
Hourly forecasts
60 minute forecasts
+ Energy / Air Quality / Agweather / Climate Normals API
Non-Commercial use only)

In [7]:
import requests
import pandas as pd
from datetime import timedelta
import time
import sys
import json
import os
from tqdm import tqdm

# --- CONFIGURATION ---

# 1. ADD YOUR API KEYS HERE
# It will rotate through this list if one key hits its rate limit.
API_KEYS = [
    "de8e274a65ec45929d501a63466da6cf", # Business Trial 1 (Expires 2025-12-03)
    "7335451fcc0646bb86611027b68292e0", # Business Trial 2 (Expires 2025-12-07)
    "9e56a4f3fab748418bc299713138f15c", # Business Trial 3 (Expires 2025-12-07)
    "fa8e8af08314f64ab0d2f7492b46e9f", # Business Trial 4 (Expires 2025-12-07)
    "680cfd56aefc45d6bfa1d40fb65a3028", # Business Trial 5 (Expires 2025-12-08)
    "a1d90659369044278f0d29c1497dfa54"  # NEW KEY (Provisioning)
]
# Global index to track which key we are currently using
CURRENT_KEY_INDEX = 1

# 2. ADD YOUR MARKETS HERE
# The script will loop through these and save a CSV for each.
MARKETS = {
    "Bardoli": {"lat": 21.1439076246341, "lon": 73.249972681491997},
    "Bardoli_Katod": {"lat": 21.12, "lon": 73.12},
    "Bardoli_Madhi": {"lat": 21.15, "lon": 73.25},
    "Kosamba": {"lat": 21.464459923454399, "lon": 72.952089010333296},
    "Kosamba_Vankal": {"lat": 21.43, "lon": 73.23},
    "Kosamba_Zangvav": {"lat": 21.48, "lon": 72.95},
    "Mahuva": {"lat": 21.097129557468101, "lon": 71.760557527893894},
    "Mahuva_Anaval": {"lat": 20.84, "lon": 73.26},
    "Mandvi": {"lat": 21.259469954916099, "lon": 73.306064794408599},
    "Nizar": {"lat": 21.47, "lon": 74.19},
    "Nizar_Kukarmuda": {"lat": 21.51, "lon": 74.13},
    "Nizar_Pumkitalov": {"lat": 21.47, "lon": 74.10},
    "Songadh": {"lat": 21.175868577082898, "lon": 73.564033008192197},
    "Songadh_Badarpada": {"lat": 21.16, "lon": 73.56},
    "Songadh_Umrada": {"lat": 21.16, "lon": 73.56},
    "Surat": {"lat": 21.193417012914299, "lon": 72.852982768754401},
    "Uchhal": {"lat": 21.17, "lon": 73.74},
    "Valod_Buhari": {"lat": 20.97, "lon": 73.31},
    "Vyara_Paati": {"lat": 21.11, "lon": 73.38},
    "Vyra": {"lat": 21.112412216859902, "lon": 73.388557572057493},
    "Amreli": {"lat": 21.559338614206901, "lon": 71.227430257825006},
    "Babra": {"lat": 21.848355060711398, "lon": 71.311297950245503},
    "Bagasara": {"lat": 21.496945117777699, "lon": 70.959329215009802},
    "Dhari": {"lat": 21.331365216974699, "lon": 71.023828198856805},
    "Rajula": {"lat": 21.0339169150099, "lon": 71.443913765569206},
    "Savarkundla": {"lat": 21.332621199094, "lon": 71.313830460040407},
    "Ahmedabad_Chimanbhai_Patal_Market_Vasana": {"lat": 22.996976855033001, "lon": 72.536392460074595},
    "Bavla": {"lat": 22.830965521422801, "lon": 72.3579378735662},
    "Dhandhuka": {"lat": 22.377117706820101, "lon": 71.978523283421794},
    "Dholka": {"lat": 22.7212505077608, "lon": 72.448905310358995},
    "Mandal": {"lat": 23.282162015159599, "lon": 71.915345532018506},
    "Sanad": {"lat": 22.997609695821499, "lon": 72.381634273951903},
    "Viramgam": {"lat": 23.125945592319901, "lon": 72.045712191155303}
}

# 3. SETTINGS
BASE_URL = "https://api.weatherbit.io/v2.0/history/daily"
START_DATE = '2024-01-01'
END_DATE = '2024-12-31'

# Columns to remove from the final CSV
COLUMNS_TO_DROP = ['snow_rate', 'weather.icon', 'revision_version', 'timestamp_utc']

# File to track progress
PROGRESS_FILE = 'scrape_progress.json'

# --- HELPER FUNCTIONS ---

def get_api_key():
    """
    Gets the current API key and advances the index for rotation.
    Returns None if all keys are exhausted.
    """
    global CURRENT_KEY_INDEX
    if CURRENT_KEY_INDEX >= len(API_KEYS):
        return None  # All keys have been tried and failed
    
    key = API_KEYS[CURRENT_KEY_INDEX]
    return key

def rotate_to_next_key():
    """
    Moves to the next key in the list.
    Returns True if a new key is available, False if all keys are exhausted.
    """
    global CURRENT_KEY_INDEX
    CURRENT_KEY_INDEX += 1
    if CURRENT_KEY_INDEX >= len(API_KEYS):
        return False  # No more keys to try
    return True

def load_progress():
    """Loads the progress file if it exists."""
    if os.path.exists(PROGRESS_FILE):
        print(f"Loading progress from {PROGRESS_FILE}...")
        try:
            with open(PROGRESS_FILE, 'r') as f:
                return json.load(f)
        except json.JSONDecodeError:
            print("Warning: Progress file is corrupt. Starting from scratch.")
            return {}
    print("No progress file found, starting from scratch.")
    return {}

def save_progress(progress):
    """Saves the current progress to the checkpoint file."""
    with open(PROGRESS_FILE, 'w') as f:
        json.dump(progress, f, indent=4)

def parse_date(date_str):
    """Converts YYYY-MM-DD string to datetime object."""
    return pd.to_datetime(date_str)

def format_date(date_obj):
    """Converts datetime object to YYYY-MM-DD string."""
    return date_obj.strftime('%Y-%m-%d')

def save_data_to_csv(new_data_frames, output_filename):
    """
    Appends new data to the CSV file, handling duplicates and headers.
    """
    if not new_data_frames:
        print("\nNo new data to save in this batch.")
        return 'SUCCESS'

    print(f"\nSaving {len(new_data_frames)} new records to CSV...")
    
    try:
        new_df = pd.concat(new_data_frames, ignore_index=True)
    except Exception as e:
        print(f"Error concatenating new dataframes: {e}")
        return 'SUCCESS' # Nothing to save

    combined_df = new_df

    # Load existing data (if any) and combine
    if os.path.exists(output_filename):
        try:
            existing_df = pd.read_csv(output_filename)
            combined_df = pd.concat([existing_df, new_df], ignore_index=True)
            print(f"Loaded {len(existing_df)} existing records. Total records before de-dupe: {len(combined_df)}")
        except pd.errors.EmptyDataError:
            print(f"Existing file {output_filename} is empty. Using new data.")
        except Exception as e:
            print(f"Error reading {output_filename}: {e}. Overwriting with new data.")
    else:
        print("Creating new file...")

    # De-duplicate, keeping the last (newest) record for any given day
    combined_df.drop_duplicates(subset=['market_name', 'datetime'], keep='last', inplace=True)
    
    # Drop unwanted columns
    existing_cols_to_drop = [col for col in COLUMNS_TO_DROP if col in combined_df.columns]
    if existing_cols_to_drop:
        combined_df.drop(columns=existing_cols_to_drop, inplace=True)
    
    # Save to CSV
    try:
        combined_df.to_csv(output_filename, index=False)
        print(f"Successfully saved {len(combined_df)} total unique records to {output_filename}")
        return 'SUCCESS'
    except PermissionError:
        print(f"\nCRITICAL: Could not save {output_filename}. Is the file open in Excel?")
        print("Please close the file and re-run the script.")
        return 'STOP_FROM_SAVE'
    except Exception as e:
        print(f"\nCRITICAL: Failed to save CSV: {e}")
        return 'STOP_FROM_SAVE'


def fetch_weather_data(market_name, lat, lon, start_date, end_date, progress):
    """
    Fetches weather data for a specific market and date range.
    Handles API key rotation and rate limiting.
    """
    
    # Use pandas to create the list of days to query
    all_days = pd.date_range(start_date, end_date, freq='D')
    total_days = len(all_days)
    
    # Get filename for this market's CSV
    output_filename = f"{market_name.replace(' ', '_')}_weather_{start_date}_to_{end_date}.csv"
    
    # This list will ONLY hold NEW data fetched in this run
    new_data_fetched_this_run = [] 
    if os.path.exists(output_filename):
        print(f"File {output_filename} already exists. New data will be appended.")
    else:
        print(f"Creating new file: {output_filename}")

    # --- Progress Loading Logic ---
    market_progress_data = progress.get(market_name, {})
    last_fetched_day_str = None
    market_status = None

    if isinstance(market_progress_data, dict):
        last_fetched_day_str = market_progress_data.get('last_fetched_day')
        market_status = market_progress_data.get('status')
    elif isinstance(market_progress_data, str):
        if market_progress_data == 'completed':
            market_status = 'completed'
            last_fetched_day_str = None
        else:
            last_fetched_day_str = market_progress_data
            print("Detected old progress format. Will update to new format.")
    
    if market_status == 'completed':
        print(f"Market {market_name} is already marked as 'completed'. Skipping.")
        return 'CONTINUE'

    # Determine where to start fetching from
    if last_fetched_day_str:
        start_fetching_from = parse_date(last_fetched_day_str) + timedelta(days=1)
        days_to_query = [day for day in all_days if day >= start_fetching_from]
        if start_fetching_from > parse_date(end_date):
            print("All days already fetched. Marking as complete.")
            days_to_query = []
        else:
            print(f"Resuming from {format_date(start_fetching_from)}...")
    else:
        days_to_query = all_days
        print(f"Starting from {start_date}...")

    # FIX 1: Handle ValueError for DatetimeIndex
    if len(days_to_query) == 0:
        print("No new days to query. Marking as complete.")
        progress[market_name] = {'status': 'completed', 'last_fetched_day': end_date}
        save_progress(progress)
        return 'CONTINUE'

    print(f"Total days to query for this market: {len(days_to_query)}.")

    # --- MAIN FETCHING LOOP (RESTRUCTURED) ---
    with tqdm(total=len(days_to_query), desc=f"Fetching for {market_name}") as pbar:
        for day in days_to_query:
            
            # This inner loop will retry the *same day* until it
            # succeeds or all keys are exhausted.
            day_fetched_successfully = False
            
            # FIX 2: Added 'while' loop for retries
            while not day_fetched_successfully:
                
                api_key = get_api_key()
                if api_key is None:
                    # All keys are exhausted for today.
                    print("\nCRITICAL: All API keys are exhausted. Stopping script.")
                    print("Saving partial data before stopping...")
                    save_status = save_data_to_csv(new_data_fetched_this_run, output_filename)
                    if save_status == 'STOP_FROM_SAVE':
                        print("Could not save partial data. Data will be re-fetched on next run.")
                    else:
                        print("Partial data saved. Progress will be updated.")
                        save_progress(progress) 
                    
                    print("Run the script again tomorrow (or add new keys) to resume.")
                    return 'STOP' # Stop the entire batch process
                
                pbar.set_description(f"Fetching for {market_name} [{day.strftime('%Y-%m-%d')}]", refresh=True)
                pbar.set_postfix_str(f"Key: ...{api_key[-4:]}")

                day_start_str = format_date(day)
                day_end_str = format_date(day + timedelta(days=1))

                params = {
                    'key': api_key,
                    'lat': lat,
                    'lon': lon,
                    'start_date': day_start_str,
                    'end_date': day_end_str
                }

                try:
                    response = requests.get(BASE_URL, params=params, timeout=10)

                    # --- Rate Limit / Auth Error ---
                    # This now correctly handles 403 (Forbidden),
                    # which your new provisioning key might return.
                    if response.status_code == 429 or response.status_code == 403:
                        print(f"\nWarning: Key ...{api_key[-4:]} failed (Rate Limit/Auth). Trying next key.")
                        if not rotate_to_next_key():
                            # This was the last key. get_api_key() will return None
                            # on the next 'while' loop iteration, triggering the STOP.
                            pass
                        
                        time.sleep(1)
                        # 'continue' will retry the 'while' loop for the SAME DAY
                        continue 

                    # --- Other HTTP Error ---
                    if response.status_code != 200:
                        print(f"\nError: Received status code {response.status_code} for {day_start_str}.")
                        print(f"Response: {response.text}")
                        time.sleep(1)
                        # Break the 'while' loop and skip to the next day.
                        # This day will be retried on the next script run.
                        break 

                    # --- Success ---
                    data = response.json()
                    daily_data = data.get('data', [])
                    
                    if daily_data:
                        df = pd.json_normalize(daily_data)
                        df['market_name'] = market_name
                        df['query_lat'] = lat
                        df['query_lon'] = lon
                        new_data_fetched_this_run.append(df)
                    
                    progress[market_name] = {'last_fetched_day': day_start_str}
                    save_progress(progress)
                    
                    day_fetched_successfully = True # This breaks the 'while' loop
                    pbar.update(1) # Advance progress bar by one day
                    time.sleep(0.1) # Be nice to the API

                except requests.exceptions.RequestException as e:
                    print(f"\nNetwork Error: {e}. Pausing for 10 seconds...")
                    time.sleep(10)
                    # 'continue' will retry the 'while' loop for the SAME DAY
                    continue
            
            # --- End of while loop ---
            # The 'for' loop will now proceed to the next day.

    # --- Market Finished ---
    save_status = save_data_to_csv(new_data_fetched_this_run, output_filename)
    if save_status == 'STOP_FROM_SAVE':
        print(f"Could not save data for {market_name}. Market will NOT be marked as complete.")
        return 'STOP' 

    progress[market_name] = {'status': 'completed', 'last_fetched_day': end_date}
    save_progress(progress)
    print(f"--- Finished processing {market_name} ---")
    return 'CONTINUE'

def main():
    """
    Main function to run the batch scraper.
    """
    print("--- Starting Weatherbit Batch Scraper ---")
    
    # Check if argparse is being used in an interactive env (like Jupyter)
    # This ignores Jupyter's internal args like --f
    if 'ipykernel_launcher.py' in sys.argv[0]:
        sys.argv = [sys.argv[0]]  # Keep only the script name

    # (argparse setup is removed as we are hardcoding markets)

    print(f"Found {len(MARKETS)} markets to process.")

    # Load progress from checkpoint file
    progress = load_progress()

    # Loop over each market defined in the config
    for market_name, coords in MARKETS.items():
        
        # --- Backwards-Compatibility Check ---
        market_progress_data = progress.get(market_name, {})
        market_status = None
        if isinstance(market_progress_data, dict):
            market_status = market_progress_data.get('status')
        elif isinstance(market_progress_data, str):
            if market_progress_data == 'completed':
                market_status = 'completed'
            else:
                market_status = None 
                print(f"Detected old progress format for {market_name}. Will update.")

        if market_status == 'completed':
            print(f"Market {market_name} is already 'completed'. Skipping.")
            continue
            
        print(f"\n--- Processing Market: {market_name} ---")
        
        status = fetch_weather_data(
            market_name=market_name,
            lat=coords['lat'],
            lon=coords['lon'],
            start_date=START_DATE,
            end_date=END_DATE,
            progress=progress # Pass the main progress object
        )
        
        if status == 'STOP':
            print("\nBatch process stopped.")
            print("Run the script again later to resume from the last checkpoint.")
            break # Stop processing other markets
    
    print("--- Batch processing finished. ---")

if __name__ == "__main__":
    main()

--- Starting Weatherbit Batch Scraper ---
Found 33 markets to process.
Loading progress from scrape_progress.json...
Market Bardoli is already 'completed'. Skipping.
Market Bardoli_Katod is already 'completed'. Skipping.
Market Bardoli_Madhi is already 'completed'. Skipping.
Market Kosamba is already 'completed'. Skipping.
Market Kosamba_Vankal is already 'completed'. Skipping.
Market Kosamba_Zangvav is already 'completed'. Skipping.
Market Mahuva is already 'completed'. Skipping.
Market Mahuva_Anaval is already 'completed'. Skipping.
Market Mandvi is already 'completed'. Skipping.
Market Nizar is already 'completed'. Skipping.
Market Nizar_Kukarmuda is already 'completed'. Skipping.
Market Nizar_Pumkitalov is already 'completed'. Skipping.

--- Processing Market: Songadh ---
File Songadh_weather_2024-01-01_to_2024-12-31.csv already exists. New data will be appended.
Resuming from 2024-08-02...
Total days to query for this market: 152.


Fetching for Songadh [2024-08-02]:   0%|          | 0/152 [00:00<?, ?it/s, Key: ...92e0]

Fetching for Songadh [2024-08-02]:   0%|          | 0/152 [00:03<?, ?it/s, Key: ...f15c]

Fetching for Songadh [2024-08-02]:   0%|          | 0/152 [00:07<?, ?it/s, Key: ...6e9f]

Fetching for Songadh [2024-08-02]:   0%|          | 0/152 [00:10<?, ?it/s, Key: ...3028]

Fetching for Songadh [2024-08-02]:   0%|          | 0/152 [00:13<?, ?it/s, Key: ...fa54]

Fetching for Songadh [2024-08-02]:   0%|          | 0/152 [00:19<?, ?it/s, Key: ...fa54]


CRITICAL: All API keys are exhausted. Stopping script.
Saving partial data before stopping...

No new data to save in this batch.
Partial data saved. Progress will be updated.
Run the script again tomorrow (or add new keys) to resume.

Batch process stopped.
Run the script again later to resume from the last checkpoint.
--- Batch processing finished. ---
